In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing
from factor_analyzer import FactorAnalyzer

# Principal Component Method

We can obtain a factor analysis using PCA. Concretely, we compute

$$
S = \frac{1}{n-1}\sum_{i=1}^{n}(\vec{X}_{i}-\bar{x})(\vec{X}_{i}-\bar{x})^{T}
$$

together with their eigenvalues and eigenvectors. Using the spectral decomposition of the true variance-covariance matrix, we get

$$
\Sigma  = \sum_{i=1}^{p}\lambda_{i}\vec{e}_{i} \simeq \sum_{i=1}^{p}\lambda_{i}\vec{e}_{i} = (\sqrt{\lambda_{1}}\vec{e}_{1} \dots \sqrt{\lambda_{m}}\vec{e}_{m})(\sqrt{\lambda_{1}}\vec{e}_{1} \dots \sqrt{\lambda_{m}}\vec{e}_{m})^{T}=\vec{L}\vec{L}^{T}
$$

In other words, we compute 

$$
l_{ij}=e_{ji}\sqrt{\lambda_{j}}
$$

# Example

We follow the example from the DATAtab YouTube video. We collected data from 20 people that participated to a survery, where they valued on a scale from 1-5 their propension to be outgoung, sociable, hard-working, dutiful, warm-hearted, and helpful. We want to know whether those characteristic can be derived from latent factors. For example, it could be that extraversion, conscientiousness, and agreeableness are latent factors of the observed data. 

In [2]:
raw_data = {'Person': [i for i in range(1, 21)],
            'Outgoing': [4, 1, 1, 4, 5, 1, 3, 2, 2, 1, 2, 4, 2, 3, 2, 2, 4, 3, 5, 2],
            'Sociable': [5, 2, 2, 5, 5, 2, 3, 2, 1, 4, 3, 5, 3, 2, 2, 4, 2, 2, 4, 1],
            'Hard-working': [4, 5, 2, 1, 5, 2, 4, 5, 1, 4, 5, 1, 2, 3, 3, 5, 3, 5, 1, 1],
            'Dutiful': [5, 4, 3, 2, 3, 3, 4, 4, 1, 4, 4, 2, 3, 2, 5, 3, 2, 1, 5, 1],
            'Warm-hearted': [4, 1, 4, 2, 4, 3, 5, 1, 1, 4, 2, 3, 4, 4, 2, 5, 1, 2, 2, 5],
            'Helpful': [3, 4, 2, 1, 3, 3, 5, 1, 4, 1, 3, 2, 5, 5, 1, 4, 1, 2, 1, 5]
           }
df = pd.DataFrame(raw_data).drop(columns="Person")
data = preprocessing.scale(df.to_numpy())

A first exploratory approach is looking at the correlation between characteristics. It seems feasible that outgoing and sociable can be generated by extraversion, hard-working and dutiful by conscientiousness, warm-hearted and helpful by agreeableness. 

In [3]:
print(np.corrcoef(data.T))

[[ 1.          0.58259059 -0.13163934 -0.0198477  -0.03791443 -0.21899519]
 [ 0.58259059  1.          0.07430592  0.32561993  0.26967815 -0.25662996]
 [-0.13163934  0.07430592  1.          0.34314085  0.00227055  0.0295083 ]
 [-0.0198477   0.32561993  0.34314085  1.          0.00139569 -0.25393885]
 [-0.03791443  0.26967815  0.00227055  0.00139569  1.          0.51955741]
 [-0.21899519 -0.25662996  0.0295083  -0.25393885  0.51955741  1.        ]]


In [4]:
pca = PCA()
pca.fit(data)
print("Percentage of total variance explained: ")
print([sum(pca.explained_variance_ratio_[ : i + 1]) for i in range(len(pca.explained_variance_ratio_))])

Percentage of total variance explained: 
[0.31212786449703916, 0.559269176839693, 0.786248189470139, 0.8985824324753666, 0.9656254527284175, 1.0]


We see that with three factors, we can explain 79% of the variance in the data. One might want to use more principles methods such as the Kaiser Criterion (i.e. looking at the eigenvalues, computed with pca.explained_variance_, greater than one), parallel analysis, or the scree-test. 

In [5]:
pca = PCA(n_components = 3)
pca.fit_transform(data)
eigenvalues = pca.explained_variance_
eigenvectors = pca.components_

In [6]:
L = np.zeros((6, 3))
for i in range(6):
    for j in range(3):
        L[i, j] = eigenvectors[j, i] * np.sqrt(eigenvalues[j])
print(L)

[[-0.69064262  0.18983359  0.54986924]
 [-0.82409753  0.48437561  0.12576364]
 [-0.15914068  0.11849107 -0.81389722]
 [-0.55049167  0.07428625 -0.67260489]
 [ 0.17600194  0.93458096 -0.00467685]
 [ 0.67517879  0.63026648 -0.02304541]]


In [10]:
communalities = np.sum(L ** 2, axis = 1)
print(communalities)

[0.8153802  0.92957296 0.70179457 0.76095686 0.90444012 0.85363332]


Or, using the built-in package, we get

In [12]:
fa = FactorAnalyzer(n_factors = 3, method = 'principal', rotation = None)
fa.fit(data)
fa.loadings_

C:\Users\soelm\Anaconda3\envs\myenv\lib\site-packages\sklearn\utils\extmath.py:368: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  warnings.warn(


array([[ 0.67315516,  0.1850269 , -0.53594624],
       [ 0.80323091,  0.47211095, -0.12257923],
       [ 0.15511115,  0.1154908 ,  0.79328888],
       [ 0.53655291,  0.07240528,  0.65557416],
       [-0.17154547,  0.91091684,  0.00455843],
       [-0.65808288,  0.61430778,  0.02246189]])

In [13]:
fa.get_communalities()

array([0.77461119, 0.88309431, 0.66670484, 0.72290902, 0.85921811,
       0.81095166])

The built-in package is numerically more stable than the implementation via PCA. However, up to the product with an orthogonal matrix (that flips the signs in the first and last column), the result is consistent. 

The computation of the component matrix has the consequence that, on the first factor, many variables highly load. This results in poor interpretation. The Varimax transformation comes to rescue, as it ensures that for every factor certain variables load as high as possible, while other variables load as low as possible. 

In [14]:
fa = FactorAnalyzer(n_factors = 3, method = 'principal', rotation = 'varimax')
fa.fit(data)
fa.loadings_

C:\Users\soelm\Anaconda3\envs\myenv\lib\site-packages\sklearn\utils\extmath.py:368: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  warnings.warn(


array([[ 0.84599278, -0.12899155, -0.20559325],
       [ 0.89773244,  0.09146983,  0.26230527],
       [-0.14932232,  0.06651026,  0.79999005],
       [ 0.20460379, -0.13562481,  0.81403453],
       [ 0.21346929,  0.89947114,  0.067828  ],
       [-0.31156595,  0.83254983, -0.14401076]])

Here it is to be recognized now that outgoing and sociable on Extraversion loads, industriously and dutiful on conscientiousness and warmheartedly and helpfully on compatibility. 